### Objectives

First, create a graph representation of the information we have. 
This would be information about the elements, and the nodes. 
Using this, we will feed it into our SVM, or a Random Forest, and see from there onwards

In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
from Generate_Excel import *
from MitoCode_Functions import open_file_dialog

In [ ]:
import pandas as pd
import igraph as ig

def Mitograph_Graph(FileName):
    # Read information from the gnet file, and create a graph object
    # generate a function that takes in the filename and gives out the graph
    G = pd.read_table(FileName + '.gnet', skiprows=1, sep='\t', names=['Source', 'Target', 'Length'])
    graph = ig.Graph.TupleList(G.itertuples(index=False), directed=False, edge_attrs=['Length'])
    layout = graph.layout_auto()  # Automatic layout calculation
    graph.vs['degree'] = graph.degree()
    # ig.plot(graph, layout=layout)
    Vol = pd.read_table(FileName + '.cc', skiprows=0, sep='\t')
    Vol.columns = ['node', 'cc', 'vol_cc']

    ids = [int(node['name']) for node in graph.vs]
    graph.vs['cc_vol'] = [Vol.loc[Vol['node'] == node, 'vol_cc'].values[0] for node in ids]
    graph.vs['cc'] = [Vol.loc[Vol['node'] == node, 'cc'].values[0] for node in ids]
    return graph, Vol

In [ ]:
# n = pd.read_csv('/home/mitosim2/Desktop/czi_files/full_node_Table_.csv')
# df = pd.read_csv('/home/mitosim2/Desktop/czi_files/full_Table_.csv')
n = pd.read_csv('/Users/birat/Onedrive/Desktop/TCDD11S/full_Table_.csv')
df = n

In [ ]:
cc_df = df[['cc_x', 'folder_name_x', 'cc_pixel_intensity_488', 'cc_pixel_intensity_405', 'cc_pixel_intensity_555', 'cc_pixel_intensity_ratio', 
            'cc_average_degree_excludeFreeEnds',
             'cc_max_PK', 'diameter', 'cc_vol_from_img_(um3)', 'branches'
            , 'nodes', 'edges']].drop_duplicates(subset = ['cc_x','folder_name_x'])

# add Percentage Length
# average edge length
# Max edge length 
# Min edge length 
# CV of the edge length 
# Average PK 

import warnings
from matplotlib import cm

def plot_cc(main_df, df_dir, title = ''):

    warnings.filterwarnings("ignore", category=UserWarning)
    # plt.clf()
    cc_df = main_df[['folder_name_x','cc_x', 'cc_length_(um)', 'cc_max_PK', 'cc_vol_from_img_(um3)', 'branches', 'nodes', 'edges','cc_pixel_intensity_405', 'cc_pixel_intensity_488', 'cc_pixel_intensity_ratio', 'cc_average_degree_excludeFreeEnds', 'diameter', 'cc_average_connectivity']].drop_duplicates()
    
    cc_df['nodes_per_length'] = cc_df['nodes'] / cc_df['cc_length_(um)']
    cc_df['edges_per_length'] = cc_df['edges'] / cc_df['cc_length_(um)']
    cc_df['avgdegreeperl'] = cc_df['cc_average_degree_excludeFreeEnds']/cc_df['cc_length_(um)']
    cc_df['avgpkperl'] = cc_df['cc_max_PK']/cc_df['cc_length_(um)']
    cc_df['average_edge_length'] = cc_df
    # cc_df = cc_df[cc_df['edges'] != 1]
    a = cc_df.groupby("folder_name_x")
    dataframelist = [group for _, group in a]


    fig, ax = plt.subplots(1, 7, figsize=(33.6, 4))
    
    fig.tight_layout(w_pad=2)
    colors = cm.tab20(np.linspace(0, 1, len(dataframelist)))
    i = -1
    for df_index, df in enumerate(dataframelist):
        edges = main_df[main_df['cc_x'] == df['cc_x'].values[0] and main_df['folder_name_x'] == df['folder_name_x'].values[0]]
        df['average_edge_length'] = edges['element_'].mean() 
        
        # ax.scatter(dat[:, 1], (datfunc),  color=colors[df_index])
        # print(len(df['cc_length_(um)']), len(df['cc_pixel_intensity_488']))
        # print(type(df['cc_length_(um)']), type(df['cc_pixel_intensity_488']))
        # # drop nan values from df 
        # # df = df.dropna()
        # print(len(df['cc_length_(um)']), len(df['cc_pixel_intensity_488']))
        ax[0].scatter(df['cc_length_(um)'] ,df['cc_pixel_intensity_ratio'],color=colors[df_index], label=df['folder_name_x'].values[0].split('hr', 1)[-1], s= 12)
        ax[1].scatter(df['nodes'],df['cc_pixel_intensity_ratio'],  color=colors[df_index], label=df['folder_name_x'].values[0].split('hr', 1)[-1], s= 12)
        ax[2].scatter(df['edges'],df['cc_pixel_intensity_ratio'],  color=colors[df_index], label=df['folder_name_x'].values[0].split('hr', 1)[-1], s= 12)
        ax[3].scatter(df['nodes_per_length'],df['cc_pixel_intensity_ratio'],  color=colors[df_index], label=df['folder_name_x'].values[0].split('hr', 1)[-1], s= 12)
        ax[4].scatter(df['edges_per_length'],df['cc_pixel_intensity_ratio'],  color=colors[df_index], label=df['folder_name_x'].values[0].split('hr', 1)[-1], s= 12)
        ax[5].scatter(df['avgdegreeperl'], df['cc_pixel_intensity_ratio'],  color=colors[df_index], label=df['folder_name_x'].values[0].split('hr', 1)[-1], s= 12)
        ax[6].scatter(df['avgpkperl'], df['cc_pixel_intensity_ratio'],  color=colors[df_index], label=df['folder_name_x'].values[0].split('hr', 1)[-1], s= 12)

    return cc_df


In [ ]:
cc_df = df[['cc_x', 'folder_name_x', 'cc_pixel_intensity_488', 'cc_pixel_intensity_405', 'cc_pixel_intensity_555', 'cc_pixel_intensity_ratio', 'cc_average_degree_excludeFreeEnds',
             'cc_max_PK', 'diameter', 'cc_vol_from_img_(um3)', 'branches'
            , 'nodes', 'edges']].drop_duplicates(subset = ['cc_x','folder_name_x'])

In [ ]:
cc_df = df[['cc_x', 'folder_name_x', 'cc_pixel_intensity_488', 'cc_pixel_intensity_405', 'cc_pixel_intensity_555', 
            'cc_pixel_intensity_ratio', 'cc_average_degree_excludeFreeEnds',
             'cc_max_PK', 'diameter', 'cc_vol_from_img_(um3)', 'branches'
            , 'nodes', 'edges']].drop_duplicates(subset = ['cc_x','folder_name_x'])

ele_df = df[['line_id', 'folder_name_x', 'element_pixel_intensity_488', 'element_pixel_intensity_405', 'element_pixel_intensity_555', 
            'element_pixel_intensity_ratio', 'element_Volume_Voxel', 'element_length_(um)', 'element_average_width' ]].drop_duplicates(subset=['line_id', 'folder_name_x'])

In [ ]:
ele_df.drop(['line_id', 'folder_name_x', 'element_pixel_intensity_555'], axis = 1)

In [ ]:
ele_corr = ele_df.drop(['line_id', 'folder_name_x', 'element_pixel_intensity_555'], axis = 1).corr()
ele_corr.to_csv('element_corr.csv')

In [ ]:
cc_corr = cc_df.drop(['cc_x', 'folder_name_x'], axis = 1).corr()
cc_corr.to_csv('cc_corr.csv')

In [ ]:
cc_df

In [ ]:
eledf = df[['line_id', 'cc_x', 'element_Volume_Voxel', 'element_pixel_intensity_405', 'element_length_(um)', 'element_pixel_intensity_488', 'element_pixel_intensity_555' 
            ,'element_pixel_intensity_ratio', 'element_average_width', 'folder_name_x']].drop_duplicates(subset=['line_id', 'folder_name_x'])

bigDf = n
# rename folder_name_x to folder_name in eledf
# eledf.rename(columns={'folder_name_x': 'folder_name'}, inplace=True)
bigDf = pd.merge(bigDf, eledf, how='left', left_on=['line_id', 'folder_name'], right_on=['line_id', 'folder_name_x'])

In [ ]:
bigDf

In [ ]:
bigDf = bigDf[['line_id', 'folder_name_x', 'node', 'node_2', 'element_pixel_intensity_488', 'element_pixel_intensity_405', 'element_pixel_intensity_555', 'element_length_(um)', 'element_average_width']]

In [ ]:
cc_df = df.drop_duplicates(subset=['cc_x', 'folder_name'])

In [ ]:
df.columns

In [ ]:
df.drop(['folder_name_x', 'line_id'], axis=1, inplace=True)

In [ ]:
df.drop('cc_pixel_intensity_488', axis=1, inplace=True)

In [ ]:
df

In [ ]:
graph = ig.Graph.TupleList(bigDf.itertuples(index=False), directed=False, edge_attrs=['line_id', 'element_pixel_intensity_ratio', 'element_pixel_intensity_405', 'element_length_(um)', 'element_pixel_intensity_488'])

# ids = [int(node['name']) for node in graph.vs]
# graph.vs['cc_vol'] = [df.loc[df['node'] == node, 'degree'].values[0] for node in ids]
# graph.vs['488'] = [df.loc[df['node'] == node, 'pixel_intensity_488'].values[0] for node in ids]
# graph.vs['405'] = [df.loc[df['node'] == node, 'pixel_intensity_405'].values[0] for node in ids]
# graph.vs['ratio'] = [df.loc[df['node'] == node, 'pixel_intensity_ratio'].values[0] for node in ids]
# graph.vs['structural'] = [df.loc[df['node'] == node, 'pixel_intensity_555'].values[0] for node in ids]

In [ ]:
df.drop_duplicates(subset=['line_id', 'folder_name_x', 'cc_x'], inplace=True)

In [ ]:
corr = df.corr()

In [ ]:
n = pd.read_csv('/home/mitosim2/Desktop/czi_files/full_node_Table_.csv')
df = pd.read_csv('/home/mitosim2/Desktop/czi_files/full_Table_.csv')

In [ ]:
cc_df = df[['cc_x', 'folder_name_x', 'cc_pixel_intensity_488', 'cc_pixel_intensity_405', 'cc_pixel_intensity_ratio',
            'cc_pixel_intensity_555',  'cc_average_degree_excludeFreeEnds',
             'cc_max_PK', 'diameter', 'cc_vol_from_img_(um3)', 'branches'
            , 'nodes', 'edges']].drop_duplicates(subset = ['cc_x','folder_name_x'])

ele_df = df[['line_id', 'folder_name_x', 'element_pixel_intensity_488', 'element_pixel_intensity_405', 'element_pixel_intensity_555', 
            'element_pixel_intensity_ratio', 'element_Volume_Voxel', 'element_length_(um)', 'element_average_width' ]].drop_duplicates(subset=['line_id', 'folder_name_x'])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# make cc_df only the first 1000 entries 
cc_df = cc_df[:1000]

In [ ]:
cc_df['cc_pixel_intensity_488'] = cc_df['cc_pixel_intensity_488'].astype(int)

In [ ]:
X = cc_df.drop(['cc_pixel_intensity_488', 'cc_pixel_intensity_405', 'cc_x', 'folder_name_x'], axis=1)  # Adjust 'target_column' to your actual target column name
y = cc_df['cc_pixel_intensity_488']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

plt.plot(y_test, y_pred, 'o')
plt.title('Accuracy: {}'.format(accuracy))
plt.xlabel('Actual')
plt.ylabel('Predicted')


In [ ]:
xx

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('test')
plt.ylabel('pred')

In [ ]:
ele_df = ele_df[:1000]
ele_df['element_pixel_intensity_488'] = ele_df['element_pixel_intensity_488'].astype(int)

X = ele_df.drop(['element_pixel_intensity_405', 'line_id', 'folder_name_x', 'element_pixel_intensity_488', 'element_pixel_intensity_ratio'], axis=1)  # Adjust 'target_column' to your actual target column name
y = ele_df['element_pixel_intensity_488']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

plt.plot(y_test, y_pred, 'o')
plt.title('Accuracy: {}'.format(accuracy))
plt.xlabel('Actual')
plt.ylabel('Predicted')

In [ ]:
# do the same for random forest
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, random_state=0)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

plt.plot(y_test, y_pred, 'o')
plt.title('Accuracy: {}'.format(accuracy))



In [ ]:
def findRepeats(x):
    for i in range(len(x)):
        if x[i] == x[i+1]:
            if i < len(x)-2:
                return 1 + findRepeats(x[i+1:])
            
    return 0

findRepeats('abababab')

In [ ]:
# 2-b 
def giveKlist(X, k): 
    nlist = []
    indexes_to_add = len(X)//k 
    for i in range(1, indexes_to_add+1):
        nlist.append(X[i*k-1])
    return nlist

giveKlist([1, 2, 3, 4, 5], 2)

In [ ]:
#2-c 
def convertBase(val, k): 
    # get val in base 1: 
    base1 = val * 10 
    # convert base1 to base k
    for i in range(0, 10): 
        if base1 < i**k: 
            break
    return i
convertBase(1, 2)

In [ ]:
#2-d 
import random 
def tossCoins(p, t): 
    # p is probability of heads 
    # t is number of tosses
    resultList = []
    for i in range(0, t): 
        if random.random() < p: 
            resultList.append('H')
        else: 
            resultList.append('T')
    return resultList
tossCoins(0.7, 10)

In [ ]:
#2-e
def printArr(A, B): 
    len1 = len(A)
    len2 = len(B)
    longer_len = max(len1, len2)
    smaller_len = min(len1, len2)
    npList = []
    for i in range(0, smaller_len): 
        npList.append(A[i])
        npList.append(B[i])
    if len1 > len2: 
        for i in range(smaller_len, longer_len): 
            npList.append(A[i])
    else:
        for i in range(smaller_len, longer_len): 
            npList.append(B[i])

    return npList        

In [ ]:
#2-f
def printMany(x, k): 
    # turn int into string  
    x = str(x)
    return k * x


In [ ]:
#2-g
def reverseString(x): 
    return x[::-1]


In [ ]:
#2-h

def reverseSkipNum(x):
    # find index position of the numbers
    indexes = []
    for i in range(len(x)): 
        if x[i].isdigit(): 
            indexes.append(i)
    print(indexes)
    newstr = ''
    # remove the numbers from the string
    for i in range(len(x)): 
        if i not in indexes: 
            newstr += x[i]
    # reverse newstr: 
    newstr = newstr[::-1]
    print(newstr)
    neweststr = ''
    k = 0
    for i in range(len(x)): 
        if i in indexes:
            neweststr += x[i]
        else: 
            neweststr += newstr[k]
            k = k + 1
    return neweststr
reverseSkipNum('a1b2c3d4e5f6')

In [ ]:
def harmonicSum(n):
    sum = 0
    for i in range(1, n+1): 
        sum += 1/i
    return sum

def inverseAdd(x, y): 
    return harmonicSum(x)/harmonicSum(y)

In [ ]:
a = '1a3rf35'
# split string at int
intList = []
for i in a: 
    if type(i) == int: 
        intList.append(i)